# CIS IEEE UNB - Renting Houce Prices Prediction

O desafio consiste em desenvolver um modelo capaz de prever o preço de imóveis do Rio de Janeiro cadastrados no AirBnB, tendo em vista as características do imóvel fornecidas durante o seu cadastro.


## Objetivos

1. Fazer uma análise do dataset utilizando ferramentas aprendidas no período
2. Demonstrar como pré-processar e representar os diferentes tipos de dados
3. Demonstrar a utilização de técnicas como one-hot encoding, redução de dimensionalidade e PCA;

    i.Como a redução da dimensionalidade afetou no desempenho do modelo?

4. Aplicar regressão nos dados - dividir o dataset em treinamento e
validação e aplicar, pelo menos, 3 algoritmos distintos de regressão para
prever o preço do imóvel. Testar visualizar os resultados, comparando o
desempenho dos modelos treinados;

    i. Qual modelo performou melhor?

5. Avaliar o desempenho do(s) modelo(s) treinado(s). A métrica da
competição será o erro quadrático médio (MSE), porém, recomenda-se a
avaliação do modelo por outras métricas adicionais, como o R2 e o erro
médio absoluto (MAE).

In [1]:
import pandas as pd
import numpy as np

test_path = './data/test.csv'
train_path = './data/train.csv'

test = pd.read_csv(test_path)
train = pd.read_csv(train_path)


/tmp/ipykernel_267765/504526654.py:8: DtypeWarning: Columns (0,21,24,29) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv(train_path)


In [2]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196030 entries, 0 to 196029
Data columns (total 36 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   id                                            196030 non-null  int64  
 1   listing_url                                   196030 non-null  object 
 2   scrape_id                                     196030 non-null  float64
 3   last_scraped                                  196030 non-null  object 
 4   name                                          195706 non-null  object 
 5   summary                                       186334 non-null  object 
 6   space                                         118777 non-null  object 
 7   description                                   191715 non-null  object 
 8   experiences_offered                           196030 non-null  object 
 9   neighborhood_overview                         10

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588091 entries, 0 to 588090
Data columns (total 37 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   id                                            588091 non-null  object 
 1   listing_url                                   588091 non-null  object 
 2   scrape_id                                     588090 non-null  float64
 3   last_scraped                                  588091 non-null  object 
 4   name                                          586971 non-null  object 
 5   summary                                       558441 non-null  object 
 6   space                                         355819 non-null  object 
 7   description                                   574567 non-null  object 
 8   experiences_offered                           588091 non-null  object 
 9   neighborhood_overview                         31

In [4]:
missing_values = train.isna().sum() / len(train) * 100
print(missing_values)

id                                               0.000000
listing_url                                      0.000000
scrape_id                                        0.000170
last_scraped                                     0.000000
name                                             0.190447
summary                                          5.041737
space                                           39.495928
description                                      2.299644
experiences_offered                              0.000000
neighborhood_overview                           46.108340
minimum_minimum_nights                          23.290613
maximum_minimum_nights                          23.290613
minimum_maximum_nights                          23.290613
maximum_maximum_nights                          23.290613
minimum_nights_avg_ntm                          23.290613
maximum_nights_avg_ntm                          23.290613
number_of_reviews_ltm                           23.290613
calculated_hos

Aqui vemos os casos criticos. 

1. Duas colunas possuem em torno de 40% de valores faltantes
2. 10 colunas possuem a mesma porcentagem de valores faltantes = 23.543845% (MCAR - Missing completely at random)

Vamos abordar da seguinte forma:

1. Excluir as colunas com aproximadamente 20% de valores faltantes
2. Imputar valores faltantes nas outras colunas com a média e moda


Excluir as colunas com aproximadamente 40% de valores faltantes para os dois dataframes.

In [5]:
# Dataframe de treino
for column in train:
    if (train[column].isna().sum() / len(train) * 100) > 20:
        train = train.drop(column,axis=1)

# Data frame de teste
for column in test:
    if (test[column].isna().sum() / len(test) * 100) > 20:
        test = test.drop(column,axis=1)

missing_values = train.isna().sum() / len(train) * 100
print(missing_values)

id                     0.000000
listing_url            0.000000
scrape_id              0.000170
last_scraped           0.000000
name                   0.190447
summary                5.041737
description            2.299644
experiences_offered    0.000000
host_is_superhost      0.049992
host_listings_count    0.049992
latitude               0.000000
longitude              0.000000
accommodates           0.000000
bathrooms              0.191467
bedrooms               0.100665
beds                   0.299103
extra_people           0.000000
minimum_nights         0.000000
number_of_reviews      0.000000
instant_bookable       0.000170
amenities              0.000000
property_type          0.000000
room_type              0.000000
cancellation_policy    0.000170
price                  0.000000
dtype: float64


Para imputar valores as colunas é preciso verificar os tipos de dados de cada uma delas.

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588091 entries, 0 to 588090
Data columns (total 25 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   588091 non-null  object 
 1   listing_url          588091 non-null  object 
 2   scrape_id            588090 non-null  float64
 3   last_scraped         588091 non-null  object 
 4   name                 586971 non-null  object 
 5   summary              558441 non-null  object 
 6   description          574567 non-null  object 
 7   experiences_offered  588091 non-null  object 
 8   host_is_superhost    587797 non-null  object 
 9   host_listings_count  587797 non-null  object 
 10  latitude             588091 non-null  float64
 11  longitude            588091 non-null  float64
 12  accommodates         588091 non-null  object 
 13  bathrooms            586965 non-null  float64
 14  bedrooms             587499 non-null  float64
 15  beds             

### Conversão de tipos

Aqui podemos veriricar certos casos onde o tipo de dado não é ótimo para uma analise de dados:

1. Dados numéricos do tipo object que deveriam ser do tipo float
2. Dados categópricos do tipo object que deveriam ser do tipo float

In [7]:
# Dados categóricos (nominal)
categorical_data = ['host_is_superhost']
numeric_data = ['minimum_nights','latitude','extra_people','longitude','host_listings_count'
                'bathrooms', 'bedrooms', 'beds', 'number_of_reviews','price','accommodates']

Alterando erros da coluna id:

In [8]:
train['id'] = range(len(train))
train['id'] = train['id'].astype('int64')

Alterando tipagem dos dados numericos e corrigindo erros de formatação.

In [9]:
train['price']

0           $229.00
1         $1,502.00
2           $569.00
3           $499.00
4           $132.00
            ...    
588086      $100.00
588087    $1,200.00
588088      $298.00
588089    $7,901.00
588090      $400.00
Name: price, Length: 588091, dtype: object

In [10]:
# Dataframe de treino
# Conversão dos dados numéricos
for column in train.columns:
    if column in numeric_data and train[column].dtype != 'float64':
        train[column] = train[column].str.replace('[\$,]', '', regex=True)
        train[column] = pd.to_numeric(train[column], errors='coerce')

# Dataframe de teste
categorical_data = ['host_is_superhost']
test_numeric_data = ['minimum_nights', 'extra_people', 'host_listings_count', 'accommodates']

for column in test.columns:
    if column in test_numeric_data and test[column].dtype != 'float64':
        test[column] = pd.to_numeric(test[column], errors='coerce')
        train[column] = pd.to_numeric(train[column], errors='coerce')
        

print(test['accommodates'])

0         5
1         3
2         3
3         1
4         2
         ..
196025    6
196026    6
196027    4
196028    3
196029    4
Name: accommodates, Length: 196030, dtype: int64


Podemos verificar agora que os dados numéricos possuem o tipo float.

### Imputação de valores faltantes

Para a imputação de valores faltantes, vamos utilizar a média para os dados numéricos e a moda para os dados categóricos.

In [11]:
# Dataframe de treino
for column in train.columns:
    if train[column].dtype == 'object': # Dados categóricos
        mode_value = train[column].mode()[0]
        train.fillna({column:mode_value}, inplace=True)
    elif train[column].dtype == 'float64': # Dados numéricos
        mean_value = train[column].mean()
        train.fillna({column:mean_value}, inplace=True)

# Dataframe de teste
for column in test.columns:
    if test[column].dtype == 'object': # Dados categóricos
        mode_value = test[column].mode()[0]
        test.fillna({column:mode_value}, inplace=True)
    elif test[column].dtype == 'float64': # Dados numéricos
        mean_value = test[column].mean()
        test.fillna({column:mean_value}, inplace=True)


test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196030 entries, 0 to 196029
Data columns (total 24 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   196030 non-null  int64  
 1   listing_url          196030 non-null  object 
 2   scrape_id            196030 non-null  float64
 3   last_scraped         196030 non-null  object 
 4   name                 196030 non-null  object 
 5   summary              196030 non-null  object 
 6   description          196030 non-null  object 
 7   experiences_offered  196030 non-null  object 
 8   host_is_superhost    196030 non-null  object 
 9   host_listings_count  196030 non-null  float64
 10  latitude             196030 non-null  float64
 11  longitude            196030 non-null  float64
 12  accommodates         196030 non-null  int64  
 13  bathrooms            196030 non-null  float64
 14  bedrooms             196030 non-null  float64
 15  beds             

Agora podemos verificar que todos os dados possuem a tipagem correta para análise a uma correta imputação desses dos valores faltantes.

In [12]:
train.isna()

,id,listing_url,scrape_id,last_scraped,name,summary,description,experiences_offered,host_is_superhost,host_listings_count,...,beds,extra_people,minimum_nights,number_of_reviews,instant_bookable,amenities,property_type,room_type,cancellation_policy,price
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
588086,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
588087,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
588088,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
588089,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### Data Encoding

Para a aplicação de modelos de regressão, é necessário que os dados estejam em formato numérico. Para isso, vamos aplicar a técnica de one-hot encoding para as variáveis categóricas. 

Aqui identificamos algumas colunas que podem ser codificadas para one-hot encoding.

1.  instant_bookable -> t/f
2.  host_is_superhost -> t/f
3.  property_type -> 48 tipos
4.  room_type -> 5 tipos
5.  cancellation_policy -> 6 tipos
6. amenities

Para os dados que possuem valores t/f utilizaremos OneHotEncoder.

Essa técnica é usada para aplicar em dados categóricos nominais. A codificação one-hot converte dados categóricos em uma matriz binária, onde cada categoria é representada por um dígito binário único (0 ou 1). Essa técnica cria uma coluna binária para cada categoria e marca a presença ou ausência dessa categoria com um 1 ou 0, respectivamente.

In [13]:
# OneHot Encoder

# Categorical data 
categoric_data = ['host_is_superhost', 'instant_bookable', 'property_type', 'room_type', 'cancellation_policy']

# Dataframe de treino
# Atribuir t -> 1, f -> 0
for column in ['host_is_superhost', 'instant_bookable']:
    train[column] = train[column].replace({'t': 1, 'f': 0})
    train[column] = train[column].infer_objects(copy=False)
    train[column] = pd.to_numeric(train[column], errors='coerce')

# Dataframe de teste
for column in ['host_is_superhost', 'instant_bookable']:
    test[column] = test[column].replace({'t': 1, 'f': 0})
    test[column] = test[column].infer_objects(copy=False)
    train[column] = pd.to_numeric(train[column], errors='coerce')

train['amenities'].unique()


/tmp/ipykernel_267765/4232557244.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train[column] = train[column].replace({'t': 1, 'f': 0})
/tmp/ipykernel_267765/4232557244.py:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test[column] = test[column].replace({'t': 1, 'f': 0})


array(['{Internet,Wifi,Kitchen,"Free parking on premises",Breakfast,"Family/kid friendly",Washer,Dryer}',
       '{TV,"Cable TV",Internet,Wifi,"Air conditioning",Kitchen,"Free parking on premises","Smoking allowed",Doorman,Gym,Elevator,"Buzzer/wireless intercom","Family/kid friendly",Washer,Dryer,"Fire extinguisher"}',
       '{TV,"Air conditioning",Pool,Kitchen,"Free parking on premises","Smoking allowed",Gym,Elevator,"Family/kid friendly","Suitable for events",Washer,Essentials,Shampoo,"Lock on bedroom door",Hangers,"Hair dryer",Iron,"Laptop friendly workspace"}',
       ...,
       '{TV,Wifi,"Air conditioning",Pool,"Free parking on premises",Breakfast,Elevator,"Hot tub",Heating,Washer,"Carbon monoxide detector","Fire extinguisher",Essentials,Shampoo,"Lock on bedroom door",Hangers,Iron,"Private entrance"}',
       '{TV,Internet,Wifi,"Air conditioning",Pool,Kitchen,"Free parking on premises","Smoking allowed","Pets allowed",Gym,Elevator,"Free street parking","Suitable for events",Wash

Para os dados categoricos com mais de 2 tipos, utilizaremos o Label Encoding (codificação de rotúlos).

A codificação de rótulos é uma técnica usada em aprendizado de máquina para converter dados categóricos em formato numérico. Ao contrário da codificação one-hot ou da codificação ordinal, a codificação de rótulos não considera necessariamente qualquer ordem ou hierarquia inerente entre as categorias. Em vez disso, ela atribui um rótulo numérico único a cada categoria, essencialmente convertendo-as em representações numéricas.

In [14]:
#Label Encoding

from sklearn.preprocessing import LabelEncoder

# Data frame de treino
for column in categoric_data[2:]:
    train[column] = LabelEncoder().fit_transform(train[column])

    
# Data frame de teste
for column in categoric_data[2:]:
    test[column] = LabelEncoder().fit_transform(test[column])

train['amenities'].nunique()

93761

### Tratando a coluna ammenities

Devido a grande quantidade de valores em ammenities, precisamos normalizar os dados dessa coluna.

In [15]:
def normalize_amenities(amenities):
    # Remover caracteres especiais e converter para minúsculas
    amenities = amenities.replace('{', '').replace('}', '').replace('"', '').lower()
    # Remover espaços em branco extras
    amenities = amenities.strip()
    return amenities

# Aplicar a função de normalização
train['amenities'] = train['amenities'].apply(normalize_amenities)

# Dividir a string de amenities em uma lista e contar o número de amenities
train['amenities_count'] = train['amenities'].str.split(',').apply(len)

print(train)

            id                            listing_url     scrape_id  \
0            0    https://www.airbnb.com/rooms/947924  2.018082e+13   
1            1   https://www.airbnb.com/rooms/2747040  2.019072e+13   
2            2  https://www.airbnb.com/rooms/13734316  2.018112e+13   
3            3  https://www.airbnb.com/rooms/30979175  2.019092e+13   
4            4   https://www.airbnb.com/rooms/3742926  2.018121e+13   
...        ...                                    ...           ...   
588086  588086  https://www.airbnb.com/rooms/17574111  2.019062e+13   
588087  588087   https://www.airbnb.com/rooms/2763759  2.018082e+13   
588088  588088  https://www.airbnb.com/rooms/11732634  2.020023e+13   
588089  588089  https://www.airbnb.com/rooms/20880236  2.019062e+13   
588090  588090   https://www.airbnb.com/rooms/3225560  2.020042e+13   

       last_scraped                                       name  \
0        2018-08-16         LARGE LOVELY ROOM GREAT FOR GROUPS   
1        2019-0

### Aplicar regressão nos dados